In [2]:
import pandas as pd
from datetime import datetime
import os
import plotly.graph_objects as go
import plotly.express as px
from sklearn.cluster import KMeans
from data_processing import clean_data, process_data, min_max_norm, z_norm, generate_features, combined_encoding

pd.options.plotting.backend = "plotly"

DATA_DIR = './data'
train = os.path.join(DATA_DIR, 'train.csv')
test = os.path.join(DATA_DIR, 'test.csv')

train_final = os.path.join(DATA_DIR, 'train_with_mrt_mall_school.csv')

train_final_df = pd.read_csv(train_final)
train_df, test_df = pd.read_csv(train), pd.read_csv(test)

train_df_cleaned = clean_data(train_df)
train_df_cleaned = process_data(train_df_cleaned)
train_df_cleaned_final = clean_data(train_final_df)
train_df_cleaned_final = process_data(train_df_cleaned_final)
# test_df_cleaned = clean_data(test_df)
# test_df_cleaned = process_data(test_df_cleaned)
train_df_cleaned_final.columns

Index(['Unnamed: 0', 'list_id', 'rent_approval_date', 'town', 'block',
       'street_name', 'flat_type', 'flat_model', 'floor_area_sqm',
       'lease_commence_date', 'latitude', 'longitude', 'subzone',
       'planning_area', 'region', 'monthly_rent', 'near_mrt_count',
       'nearest_code', 'nearest_mrt_name', 'nearest_open_year',
       'nearest_mrt_dist', 'near_mall_count', 'nearest_mall_name',
       'nearest_mall_dist', 'near_school_count', 'nearest_school_name',
       'nearest_school_dist', 'type_model', 'model_digit', 'type_digit',
       'psqm', 'date', 'age'],
      dtype='object')

## MRT Count and Distance correlation with psqm price

### Region level

In [3]:
train_df_cleaned_final.groupby(['rent_approval_date', 'region'])[['psqm', 'near_mrt_count']].corr().iloc[0::2,-1].reset_index().plot.line(x='rent_approval_date', y='near_mrt_count', color='region')

In [4]:
train_df_cleaned_final.groupby(['rent_approval_date', 'region'])[['psqm', 'nearest_mrt_dist']].corr().iloc[0::2,-1].reset_index().plot.line(x='rent_approval_date', y='nearest_mrt_dist', color='region')

In [33]:
train_df_cleaned_final['psqm_rank_perc'] = train_df_cleaned_final.groupby(['region', 'rent_approval_date'])[['psqm']].rank(pct=True)

In [34]:
train_df_cleaned_final['dist_rank_perc'] = train_df_cleaned_final.groupby(['region', 'rent_approval_date'])[['nearest_mrt_dist']].rank(pct=True)

In [36]:
train_df_cleaned_final.groupby(['rent_approval_date', 'region'])[['psqm_rank_perc', 'dist_rank_perc']].corr().iloc[0::2,-1].reset_index().plot.line(x='rent_approval_date', y='dist_rank_perc', color='region')

In [74]:
from sklearn.linear_model import LinearRegression
df = train_df_cleaned_final
# Fit a simple linear regression model
model = LinearRegression()
model.fit(df[['nearest_mrt_dist', 'dist_rank_perc']], df['psqm'])

# Calculate predicted values and residuals
df['predicted'] = model.predict(df[['nearest_mrt_dist', 'dist_rank_perc']])
df['residuals'] = df['psqm'] - df['predicted']

# Plot residuals
df.plot.scatter(x="nearest_mrt_dist", y="residuals", 
                 title="Residual Plot",
                 template="plotly_white")
df['residuals'].mean()

3.0430176896819225e-15

In [73]:
from sklearn.linear_model import LinearRegression
df = train_df_cleaned_final
# Fit a simple linear regression model
model = LinearRegression()
model.fit(df[['nearest_mrt_dist']], df['psqm'])

# Calculate predicted values and residuals
df['predicted'] = model.predict(df[['nearest_mrt_dist']])
df['residuals'] = df['psqm'] - df['predicted']

# Plot residuals
df.plot.scatter(x="nearest_mrt_dist", y="residuals", 
                 title="Residual Plot",
                 template="plotly_white")
df['residuals'].mean()

2.1221543041368327e-15

In [59]:
# df['date']

0        1.630426e+09
1        1.651334e+09
2        1.664554e+09
3        1.627747e+09
4        1.667232e+09
             ...     
59995    1.630426e+09
59996    1.680278e+09
59997    1.654013e+09
59998    1.672502e+09
59999    1.675181e+09
Name: date, Length: 60000, dtype: float64

### Town level

In [21]:
train_df_cleaned_final.groupby(['rent_approval_date', 'town'])[['psqm', 'near_mrt_count']].corr().iloc[0::2,-1].reset_index().plot.line(x='rent_approval_date', y='near_mrt_count', color='town')

In [22]:
train_df_cleaned_final.groupby(['rent_approval_date', 'town'])[['psqm', 'nearest_mrt_dist']].corr().iloc[0::2,-1].reset_index().plot.line(x='rent_approval_date', y='nearest_mrt_dist', color='town')

If at that month, only have one rent, it will be NaN for corr 